In [1]:
import os
import selenium
# 파이썬과 달리 conda prompt에서 conda install selenium로 설치 후 이용
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import pandas as pd
# NameError: name 'Keys' is not defined 가 뜨므로 keys를 import해와야한다

<h2>위도 경도 반환을 위해 가게의 정확한 주소명을 알고자 한다.<br>
이를 위해 기존에 있는 불확실한 주소명과 가게명을 합쳐, <br>카카오맵에 검색해, 거기 있는 주소를 얻을 계획(이었다).

In [2]:
df = pd.read_excel('가맹점 위도경도(실패).xlsx', 'Sheet1')
df["주소"]

0      부산 해운대구 반여1동 1639-3 대룡빌딩 1010410홍콩반점
1                       부산광역시 강서구 식만로가락한식뷔페
2                          부산광역시 강서구 제도로가야정
3              부산광역시 사하구 감내2로 (감천동 1층)감내비빔밥
4                   부산광역시 강서구 낙동북로강동농협하나로마트
                       ...                 
470                       부산광역시 동래구 석사북로화성루
471                      부산광역시 강서구 제도로황금오리알
472                       부산광역시 남구 용주로황성루반점
473                     부산광역시 사하구 하신번영로황조김밥
474                    부산광역시 금정구 부곡로황태자베이커리
Name: 주소, Length: 475, dtype: object

우선 불확실한 주소명과 가게명을 합친 엑셀 파일을 불러온다.<br>
카카오맵의 경우 검색을 해도 url상의 변경이 나타나지 않는다.<br>
따라서 값을 넣은 후, selenium을 통해 자동적으로 검색을 시행해, BeautifulSoup으로 크롤링을 해서 주소를 얻어와야한다.

In [3]:
clist = ['사직동 스타벅스']
options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
options.add_argument('headless') # 브라우저 안 띄우기

chromedriver_path = r"C:\Users\82105\Desktop\map\chromedriver" # 크롬 드라이버 위치(chromdriver앱이 있는 곳)
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

driver.get("https://map.kakao.com/")  # 크롬 드라이버에 띄울 첫 페이지

search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]') # 검색 창 찾아오기

for i in clist:
    search_area.send_keys(i)  # 검색어 입력
    
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    cafe_lists = soup.select('div.keywordSearch div.sectiontitle')
    # 왜 두녀석 사이를 띄웠느냐?? 'div.keywordSearch > ul.placelist'는 자식 선택자! 즉 바로 밑에 있는 tag만 선택할 수 있음
    # 반면 띄어쓰기하면? 자손 선택자! 바로 밑뿐만 아니라 하위의 모든 tag를 선택할 수 있음

    # cafe_lists = soup.select('.body > .keywordSearch > .section placeslst > .placelist')
    print(cafe_lists)
    
    search_area.clear()
    # 검색을 여러 번 수행할 경우, 검색 창을 다시 초기화한다     
    
    # for cafe in cafe_lists:
    #     cafe_info = cafe.select('li')
    #     print(cafe_info[0].text)
    
    # 가게 주소는 카카오맵 상 KeywordSearch -> section place lst -> placelist -> placeitem -> info-item -> addr에 존재하는데...
    # 아무리 시도해봐도 placelist까지의 내용만 받아와지고, 그 하위 항목들이 받아와지지 않는다. 이럴수가!    
    

    driver.quit() # driver 종료, 브라우저 닫기

[<div class="sectiontitle">
<h5 class="themetit">테마</h5>
<span class="cntwrap"><em class="cnt" id="info.search.theme.cnt"></em></span>
</div>, <div class="sectiontitle sectionBus">
<h5 class="busestit">버스번호</h5>
<span class="cntwrap"><em class="cnt" id="info.search.buses.cnt"></em></span>
<span class="busstop HIDDEN" id="info.search.buses.busstop">
                    버스 정류장<em class="cnt" id="info.search.buses.busstop.cnt"></em>
</span>
<div class="regionsbox" id="info.search.buses.regions"></div>
</div>, <div class="sectiontitle sectionBus">
<h3 class="busstoptitle">버스 정류장</h3>
<span class="cntwrap"><em class="cnt" id="info.search.busstops.cnt"></em></span>
<div class="regionsbox" id="info.search.busstops.regions"></div>
</div>, <div class="sectiontitle">
<h3 class="addrtitle">주소</h3>
<span class="cntwrap"><em class="cnt" id="info.search.address.cnt"></em></span>
</div>, <div class="sectiontitle">
<h5 class="placetit">장소</h5>
<span class="cntwrap"><em class="cnt" id="info.search.plac